In [1]:
import json
import random
import os
import shutil
import socket
from tqdm import tqdm
import pandas as pd


In [9]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
udev            16457900        0  16457900   0% /dev
tmpfs            3293880     8784   3285096   1% /run
/dev/sda1       50633320 33322272  17294664  66% /
tmpfs           16469388        0  16469388   0% /dev/shm
tmpfs               5120        0      5120   0% /run/lock
tmpfs           16469388        0  16469388   0% /sys/fs/cgroup
/dev/sda15        106858     3704    103155   4% /boot/efi
tmpfs            3293880        0   3293880   0% /run/user/1002


In [2]:
in_nimble = False
in_WSL = False
in_tpu = False
in_pi = False

host = socket.gethostname()
if('cuda' in host):
    in_nimble = True
    print("In NimbleBox")
if(host == 'LTsuphale-NC2JM'):
    in_WSL = True
    print("In WSL")    
if(host == 'raspberrypi'):
    in_pi = True
    print("In raspberry-pi")    

In [14]:
copy_required = False
# total_test_images = 118287
total_test_images   = 5000

if(in_WSL == True):
    coco_dir = '/home/suphale/snehal_bucket/coco/raw-data'
    test_image_dir = '/home/suphale/WorkSpace/test_images'
#     test_image_dir = train_2017_dir
    test_image_captions_txt = test_image_dir + '/captions.txt'
if(in_nimble == True):
    coco_dir = '/mnt/disks/user/project/coco'
    # test_image_dir = '/mnt/disks/user/project/WorkSpace/test_images'
if(in_nimble == True):
    coco_dir = '/mnt/disks/user/project/coco'
    # test_image_dir = '/mnt/disks/user/project/WorkSpace/test_images'

coco_dir = '/home/suphale/coco/downloads'
annotation_dir = coco_dir + '/annotations'
train_2017_dir = coco_dir + '/train2017'
test_image_dir = train_2017_dir
captions_val2017_json = annotation_dir + '/captions_val2017.json'
captions_train2017_json = annotation_dir + '/captions_train2017.json'
instances_train2017_json = annotation_dir + '/instances_train2017.json'
instances_val2017_json = annotation_dir + '/instances_val2017.json'

test_image_captions_txt = './lists/captions_' + str(total_test_images) + '.txt'
test_image_list_txt = './lists/images_' + str(total_test_images) + '.txt'
test_instances_list_txt = './lists/instances_' + str(total_test_images) + '.txt'
if(test_image_dir == train_2017_dir):
    copy_required = False
print(test_image_captions_txt)

./lists/captions_5000.txt


In [15]:
f = open(instances_val2017_json,)
data_instances = json.load(f)

In [16]:
f = open(captions_val2017_json,)
data_captions = json.load(f)

In [17]:
if(copy_required == True):
    shutil.rmtree(test_image_dir,ignore_errors = True)
    os.mkdir(test_image_dir)
captions_file = open(test_image_captions_txt, "w")
image_list_file = open(test_image_list_txt, "w")
instances_list_file = open(test_instances_list_txt, "w")

In [18]:
total_images = len(data_captions['images'])
print("total_images = %d" % (total_images))
print("total_test_images = %d" % (total_test_images))
df_caption = pd.DataFrame(columns = ['FileName' , 'Caption']) 
df_annotation = pd.DataFrame(columns = ['FileName' , 'Caption']) 

if (total_images < total_test_images):
    total_test_images = total_images
print("total images = %d total test images = %d" % (total_images,total_test_images))
random_list = random.choices(data_captions['images'],k=total_test_images)
for x in tqdm(random_list):
    file_name= x['file_name']
    image_id = x['id']
    image_list_file.write(file_name+'\n')
    if(copy_required == True):
        shutil.copyfile(train_2017_dir + '/' + file_name, test_image_dir + '/' + file_name)
    for a in data_captions['annotations']:
        if(a['image_id'] == image_id):
            captions_file.write(file_name + ',' + a['caption']+'\n')
            series_obj = pd.Series( [file_name, a['caption']], index=df_caption.columns )
            df_caption = df_caption.append( series_obj, ignore_index=True)
    for a in data_instances['annotations']:
        if(a['image_id'] == image_id):
            instances_list_file.write(file_name + ',' + str(a['category_id'])+'\n')
            series_obj = pd.Series( [file_name, a['category_id']], index=df_annotation.columns )
            df_annotation = df_annotation.append( series_obj, ignore_index=True)

captions_file.close()
image_list_file.close()
instances_list_file.close()
df_caption.to_csv('df_caption.csv')
df_annotation.to_csv('df_annotation.csv')


  0%|          | 9/5000 [00:00<01:55, 43.20it/s]

total_images = 5000
total_test_images = 5000
total images = 5000 total test images = 5000


100%|██████████| 5000/5000 [02:16<00:00, 36.68it/s]


In [19]:
# data['images'][0]

In [20]:
df_categories = pd.DataFrame(columns = ['id' , 'name'])
for x in data_instances['categories']:
    series_obj = pd.Series( [x['id'], x['name']], index=df_categories.columns )
    df_categories = df_categories.append( series_obj, ignore_index=True)
df_categories.to_csv('categories.csv')

In [21]:
df_caption

,FileName,Caption
0,000000213224.jpg,A vase filled with flowers next to a smaller v...
1,000000213224.jpg,Two vases filled with flowers on a table.
2,000000213224.jpg,Two vases of fresh flowers sit on top of the t...
3,000000213224.jpg,a small vase and a large vase of flowers on a ...
4,000000213224.jpg,A wooden table holding two vases of flowers.
...,...,...
25006,000000349860.jpg,A BOY WITH A BLUE SHIRT AND JEAN PANTS DOING A...
25007,000000349860.jpg,a young man doing a jump with his skateboard i...
25008,000000349860.jpg,A man has jumped a ramp with his skateboard.
25009,000000349860.jpg,a man in the air with a skateboard above a hal...
